This code outlines the implementation of a **Memorizing Transformer** model, which incorporates **K-Nearest Neighbors (KNN)** memory retrieval and **relative position encoding** into a Transformer-like architecture. I'll walk you through the important parts of the code to help you understand what's going on, step-by-step.

### Key Concepts:

1. **Transformer Architecture**: This is a deep learning model used primarily for NLP tasks, where words (or tokens) are passed through layers of attention mechanisms. Transformers use "self-attention" to process all tokens simultaneously, allowing them to capture long-range dependencies in sequences.

2. **Relative Position Encoding**: Rather than using absolute positions (fixed numbers that represent token positions), this model uses relative positions between tokens. This helps the model to handle variable-length sequences better.

3. **K-Nearest Neighbors (KNN)**: In this context, KNN is used to retrieve relevant "memories" from previous computations. This means that instead of only considering the current input sequence, the model can also access stored memory (previous computations or knowledge), which is used to augment the attention mechanism.

### High-Level Overview of Workflow:

1. **Tokenization**: Raw text is tokenized into tokens using `AutoTokenizer`.
2. **Embedding**: Token IDs are passed through an embedding layer to convert them into continuous vectors.
3. **Layer Processing**: The input passes through multiple Transformer blocks:
   - **Attention**: Captures relationships between tokens using self-attention (or KNN-enhanced attention).
   - **Feedforward**: Further processes the attention outputs.
4. **Memory Augmentation**: Each layer may augment its attention with memories from the KNN structure.
5. **Output**: The final output is passed through a linear layer to generate token probabilities, and the loss is calculated using these predictions.

### Why This Approach is Important:

- **Relative Position Encoding**: This allows the model to handle long sequences and variable lengths without needing to explicitly memorize the position of each token in the sequence. It can generalize better over different lengths.
  
- **KNN Memory**: The addition of KNN memory helps the model "remember" past computations, making it more powerful for long-term sequence generation or tasks where context from distant tokens is necessary.

### Key Terms Explained:

- **Attention**: A mechanism in neural networks where each output token is computed as a weighted sum of input tokens. The weights (attention scores) are learned during training.
  
- **KNN**: A machine learning algorithm that stores a set of "memories" (previous computations) and retrieves the closest ones based on a distance metric.
  
- **Cross-entropy loss**: A loss function used for classification problems, comparing the predicted probabilities with the true labels.
  
- **Residual connections**: Skip connections that allow the model to retain previous layer outputs, which helps mitigate issues like vanishing gradients during training.

Let me know if you would like any part of this explained further!

#### **Imports**

- `torch`, `torch.nn`, and `torch.optim`: Standard PyTorch libraries for defining neural networks and optimization.
- `transformers` and `AutoTokenizer`: Used to load and preprocess data (e.g., converting text into tokens).
- `faiss`: A library used for efficient similarity search and clustering (useful for KNN memory retrieval).
- `datasets`: A library for managing datasets.
- `einops`: A powerful utility for tensor manipulation, used to rearrange, repeat, or unpack tensors efficiently.

In [1]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import math
import os
import random
import tqdm
import gzip

!pip install einops
from einops import rearrange, repeat, pack, unpack, einsum
from einops.layers.torch import Rearrange


from functools import partial, wraps
from contextlib import contextmanager, ExitStack
from pathlib import Path
from filelock import FileLock
import pickle

import transformers
from transformers import AutoTokenizer

!pip install faiss-gpu
import faiss

!pip install datasets
import datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Fix up Classes

#### **RelativePosition Class**
This class is responsible for creating **relative position embeddings**.

- `relative_position_bucket`: This function computes relative positions between tokens in the sequence and maps them to predefined buckets. It normalizes positions and uses a logarithmic scale to handle longer sequences more effectively.
  
- `forward`: This function calculates the position of each token relative to others and returns an embedding vector for each relative position.

In [3]:
class RelativePosition(nn.Module):
  def __init__(
      self,
      rp_scale,
      num_buckets = 32,
      rp_max_distance = 128,
      heads = 8
  ):
      super().__init__()
      self.scale = rp_scale
      self.num_buckets = num_buckets
      self.rp_max_distance = rp_max_distance
      self.relative_attention_embedding = nn.Embedding(num_buckets, heads)

  def relative_position_bucket(self, relative_position_matrix):
      n = -relative_position_matrix
      n = torch.max(n, torch.zeros_like(n))

      max_exact = self.num_buckets // 2

      is_small = n < max_exact
      val_if_large = max_exact + (torch.log(n.float() / max_exact) / math.log(self.rp_max_distance / max_exact) * (self.num_buckets - max_exact)).long()
      val_if_large = torch.min(val_if_large, torch.full_like(val_if_large, self.num_buckets - 1))

      return torch.where(is_small, n, val_if_large)

  def forward(self, sequence_length):

      # Change: In the new version, context_pos is created with 2 * sequence_length, and the positions are rearranged using rearrange() to create a more explicit relation between sequence and context positions.
      # Reason: The change is likely made to increase the context size relative to the sequence length, allowing the model to process larger contexts and better capture long-range dependencies. The use of rearrange clarifies how the positions are expanded in dimensions for later computations.
      sequence_pos = torch.arange(sequence_length, dtype=torch.long)
      #########
      context_pos = torch.arange(2 * sequence_length, dtype=torch.long)
      sequence_rel_pos = rearrange(sequence_pos, 'i -> i 1')
      context_rel_pos = rearrange(context_pos, 'j -> 1 j')
      rel_pos = context_rel_pos - sequence_rel_pos

      position_bucket_indices = self.relative_position_bucket(rel_pos)

      rp_values = self.relative_attention_embedding(position_bucket_indices)

      # Change: In the new version, rearrange() is used instead of transpose() and unsqueeze(). It reformats the rp_values tensor from its original dimensions (i, j, h) to ((), h, i, j), where () implies the batch dimension is empty or not used explicitly.
      # Reason: This change might be made to clarify or standardize the dimensionality of the output tensor, ensuring that the relative position embeddings align correctly with the model's expected shape. The rearrange operation here likely facilitates batch processing or ensures the correct dimensionality for multi-head attention.
      rp_values = rearrange(rp_values, 'i j h -> () h i j')
      return rp_values * self.scale



- Change:

In the new version, context_pos is created with 2 * sequence_length, and the positions are rearranged using rearrange() to create a more explicit relation between sequence and context positions.

- Reason:

The change is likely made to increase the context size relative to the sequence length, allowing the model to process larger contexts and better capture long-range dependencies. The use of rearrange clarifies how the positions are expanded in dimensions for later computations.

In [4]:
## Old

sequence_length = 5
sequence_pos = torch.arange(sequence_length, dtype=torch.long)
context_pos = torch.arange(sequence_length, dtype=torch.long)
context_pos = torch.arange(-sequence_length, sequence_length, dtype=torch.long)
sequence_rel_pos = rearrange(sequence_pos, 'i -> i 1')
context_rel_pos = rearrange(context_pos, 'j -> 1 j')
rel_pos = context_rel_pos - sequence_rel_pos
rel_pos

tensor([[-5, -4, -3, -2, -1,  0,  1,  2,  3,  4],
        [-6, -5, -4, -3, -2, -1,  0,  1,  2,  3],
        [-7, -6, -5, -4, -3, -2, -1,  0,  1,  2],
        [-8, -7, -6, -5, -4, -3, -2, -1,  0,  1],
        [-9, -8, -7, -6, -5, -4, -3, -2, -1,  0]])

In [5]:
## Improved

sequence_length = 5
sequence_pos = torch.arange(sequence_length, dtype=torch.long)
context_pos = torch.arange(2*sequence_length, dtype=torch.long)
#context_pos = torch.arange(-sequence_length, sequence_length, dtype=torch.long)
sequence_rel_pos = rearrange(sequence_pos, 'i -> i 1')
context_rel_pos = rearrange(context_pos, 'j -> 1 j')
rel_pos = context_rel_pos - sequence_rel_pos
rel_pos

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [-1,  0,  1,  2,  3,  4,  5,  6,  7,  8],
        [-2, -1,  0,  1,  2,  3,  4,  5,  6,  7],
        [-3, -2, -1,  0,  1,  2,  3,  4,  5,  6],
        [-4, -3, -2, -1,  0,  1,  2,  3,  4,  5]])

#### **KNN_XLAttention Class**
This class defines a custom attention mechanism called **KNN Attention**. It combines traditional self-attention with KNN retrieval to enhance the model's ability to memorize long-term dependencies.

- **Input Tensors**:
  - `queries`, `keys`, and `values`: These are standard components of the attention mechanism. Queries are compared to keys to calculate attention scores, which are then used to weight the values.
  
- **KNN Memory Retrieval**:
  - The `knn.search` function searches for the closest memories (key-value pairs) to the current query. These memories are then integrated into the attention mechanism.

- **Masking**:
  - A triangular mask is applied to ensure that the model does not attend to future tokens (important for causal language modeling).

- **Output**:
  - The attention outputs are combined with the KNN-based outputs, weighted by a gate bias.

In [6]:
class KNN_XLAttention(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        knn,
        heads = 8,
        head_dimension = 32,
        topk_retrieved_memories = 3,
        dropout = 0.
    ):
        super().__init__()
        self.heads = heads
        self.scale = head_dimension ** -0.5

        # Change: A dropout parameter is introduced, which is used to apply dropout to the attention scores and softmax results.
        # Reason: Dropout is commonly used to prevent overfitting and improve generalization. By introducing a configurable dropout rate, the model can be adapted for different regularization needs.
        self.dropout = nn.Dropout(dropout)

        self.query_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.output_matrix = nn.Linear(heads * head_dimension, embedding_dimension)

        self.gate_bias = nn.Parameter(torch.randn(self.heads, 1, 1))
        self.topk_retrieved_memories = topk_retrieved_memories
        self.knn = knn

    def forward(
        self,
        x, # batch_size, sequence_length, embedding_dimension

        # Change: The new version introduces a relative_positions argument in the forward method.
        # Reason: This change enables the model to incorporate relative position encodings, which can help the model better capture long-range dependencies and improve the effectiveness of attention over sequences with varying lengths.
        relative_positions = None,
        xl_memory = None
    ):
        batch_size, sequence_length = x.shape[:2]
        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        # Change: In the new version, F.normalize is applied to the queries and keys along the last dimension.
        # Reason: Normalization helps to stabilize training by ensuring that the inputs to the attention mechanism are on a consistent scale. This can also help with convergence speed and overall model performance.
        queries = F.normalize(queries, dim=-1)
        keys = F.normalize(keys, dim=-1)

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim = -2) # unstack
            keys = torch.cat((k_xl, keys), dim = -2) # prepend XL memory
            values = torch.cat((v_xl, values), dim = -2) # prepend XL memory
            xl_sequence_length = k_xl.shape[1]

        ### LOCAL ATTENTION

        queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
        keys    = rearrange(keys, 'b t (h d) -> b h t d', h = self.heads)
        qk      = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        qk = qk * self.scale

        # Change: The new version adds the relative_positions information to the attention scores (qk), but only if relative_positions is provided.
        # Reason: The addition of relative position encoding enhances the model's ability to understand the relative position between different elements in the sequence. This is especially useful in tasks that require long-term dependencies or in transformers that are sensitive to sequence length and order.
        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk



        mask = torch.ones((i,j), dtype = torch.bool).triu(j-i+1)
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)

        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk@values
        qkv = rearrange(qkv, 'b h t d -> b t (h d)')

        ### KNN ATTENTION

        # Convert queries to search form
        queries = rearrange(queries, 'b h t d -> b t (h d)')
        mem_kv = knn.search(queries, topk = self.topk_retrieved_memories) # returns b t k 2 d
        mem_k, mem_v = mem_kv.unbind(dim = -2)
        mem_k = rearrange(mem_k, 'b t k (h d) -> b h t k d', h=self.heads)
        mem_v = rearrange(mem_v, 'b t k (h d) -> b h t k d', h=self.heads)

        # Convert queries to attention form
        queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
        mem_qk = einsum('b h t d, b h t k d -> b h t k', queries, mem_k)
        mem_qk = mem_qk * self.scale

        mem_qk = F.softmax(mem_qk, dim=-1)
        mem_qk = self.dropout(mem_qk)
        mem_qkv = einsum('b h t k, b h t k d -> b h t d', mem_qk, mem_v)

        # Combined attentions
        # Change: The gate bias (self.gate_bias) is used to combine the local and KNN-based attention results, controlling how much influence each should have on the final output.
        # Reason: The gate bias introduces a learnable weighting mechanism between the two attention types (local and KNN), potentially improving the model's ability to blend these two sources of information in a task-specific way.
        combined_qkv = mem_qkv * self.gate_bias + qkv * (1 - self.gate_bias)
        combined_qkv = rearrange(combined_qkv, 'b h t d -> b t (h d)')
        out = self.output_matrix(combined_qkv)

        # New XL memories
        keys = rearrange(keys, 'b h t d -> b t (h d)', h = self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2) # (batch, sequence_len, 2, dimension)

        if xl_memory is not None:
            # if we're on a middle/end segment of a document (there are previous XL memories)
            xl_memories, current_kv = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
        else:
            # if we're at the first segment
            current_kv = kv_memories

        knn.add(current_kv)

        return out, current_kv

Let's build the model!

From the paper:

"The input text is tokenized, and the tokens
are embedded into vector space. The embedding vectors are passed through a series of transformer
layers, each of which does dense self-attention, followed by a feed-forward network (FFN). Since
this is a decoder-only language model, we use a causal attention mask and the token embeddings of
the last layer are used to predict the next token."


This code outline demonstrates the basic flow of a neural network model for natural language processing (NLP), possibly a transformer-based architecture like GPT or BERT. Here's a breakdown of each section:

### 1. **Embedding**:
- **Explanation**: The raw text is first tokenized using the `tokenizer`, which converts the text into a series of token IDs. These IDs represent words or subwords in a vocabulary. The `embedding` layer then transforms these token IDs into dense vector representations (embeddings), which are used as the initial input for the model.

### 2. **BLOCK (n layers)**:
This section indicates that the model consists of `n` layers of blocks. Each block typically contains an attention mechanism followed by a feedforward network, and each block operates on the output of the previous one.

### 3. **Attention**:
  - **Residual Connection**: A copy of the input `x` is saved as `residual` before any transformation. This is part of the **residual connection**, a technique introduced in ResNet (and used in transformers) where the original input is added back after the transformation to help with gradient flow and improve convergence during training.
  - **Layer Normalization**: Before applying attention, `x` is normalized using layer normalization. Layer normalization standardizes the inputs to each layer, helping stabilize training.
  - **Attention**: The core mechanism here is the attention layer, which could be **XL (Extended Memory) attention** or **KNN_XL** (K-nearest neighbors with extended memory). These attention mechanisms allow the model to focus on different parts of the input sequence, with KNN_XL additionally enabling the model to retrieve and attend to past memory sequences for better long-term dependency modeling.
  - **Residual Addition**: After the attention transformation, the original input (stored in `residual`) is added back to the result. This helps preserve important features from the input that might be lost in transformations.

### 4. **Feedforward**:
  - **Residual Connection**: As with the attention part, a copy of `x` is saved as `residual` to be added back later.
  - **Layer Normalization**: The input is normalized again to ensure stable training before passing it through the feedforward network.
  - **Linear Layers**: The `linear` and `linear_2` layers are fully connected layers that transform the input data to higher or lower dimensions.
  - **Activation Function**: After the first linear transformation, a nonlinear activation function (like ReLU, GELU, etc.) is applied to introduce nonlinearity into the model, enabling it to learn more complex functions.
  - **Dropout**: Dropout is applied to prevent overfitting. It randomly zeros out a fraction of the neurons during training to reduce reliance on any single neuron.
  - **Residual Addition**: As in the attention section, the original input to the feedforward block (`residual`) is added back to the transformed output to form the final result. This is another residual connection.

### 5. **Output**:
  - **Layer Normalization**: The final output `x` is normalized before being passed to the output layer. This ensures that the output has stable values and is well-scaled.
  - **Embedding Reverse**: The output embeddings are transformed back into token IDs using the `embedding_reverse` layer. This step converts the model's learned representation back into a form that can be compared to the target output (token IDs).
  - **Loss Calculation**: The model's predicted token IDs are compared to the true `labels` using the `cross_entropy` loss function. Cross-entropy is commonly used in classification tasks, where the model is trying to predict the correct label from a set of possible classes (in this case, token IDs). The loss is minimized during training, which helps the model improve its performance over time.

### **Summary of the Process**:
- **Input**: The raw text is tokenized and embedded.
- **Attention Mechanism**: The model processes the embeddings using a combination of attention (XL or KNN_XL) and residual connections.
- **Feedforward**: The output from attention is further processed through a feedforward network with normalization, linear transformations, activation, and dropout.
- **Output**: Finally, the output is transformed back into token IDs, and a loss is computed using cross-entropy.

The overall structure resembles that of transformer-based architectures (like GPT, BERT), where attention mechanisms allow the model to focus on different parts of the sequence and feedforward networks help to capture complex patterns in the data. The use of residual connections and layer normalization helps ensure stable and efficient training.

In [ ]:
# Outline

# Embedding
token_ids = tokenizer(raw_text)
x = embedding(token_ids)

# BLOCK x n (layers)

# Attention
residual = x.copy()
x = layernorm(x)
x = attention(x) # XL, KNN_XL
x = x + residual

# Feedforward
residual = x.copy()
x = layernorm(x)
x = linear(x)
x = activation(x)
x = dropout(x)
x = linear_2(x)
x = x + residual


# Output
x = layernorm(x)
token_ids = embedding_reverse(x)
loss = cross_entropy(token_ids, labels)

In [ ]:
# build a pseudocode version of things first
# build the simplest version possible
# keep a checklist
# test

#### **Block Class**
This class defines a standard block (layer) in the Transformer model, consisting of **Attention** and **Feedforward** sub-blocks.

- **Attention**:
  - The input is passed through attention layers (either XL or KNN-based). Residual connections (additive shortcuts) are used to stabilize learning.
  
- **Feedforward Network**:
  - After the attention step, the output goes through a feedforward network (linear layers with activation functions) to further transform the representations.

In [9]:
class Block(nn.Module):
    def __init__(self, knn=None, dropout=0.):
        if knn:
            self.attention = KNNAttention(self.embedding_dimension,
                            knn,
                            heads = self.heads,
                            head_dimension = self.head_dimension,
                            dropout = self.dropout)
        else:
            self.attention = XLAttention(self.embedding_dimension,
                            heads = self.heads,
                            head_dimension = self.head_dimension,
                            dropout = self.dropout)

        self.ff_block = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim))

    def forward(self, x, xl_memories, rel_pos):
        residual = x
        norm = nn.LayerNorm(dim)
        attn_out = norm(x)
        attn_out, new_xl_memories = self.attention(attn_out, relative_positions=rel_pos, xl_memory=xl_memories)
        attn_out += residual

        residual = attn_out
        ff_out = self.ff_block(attn_out)
        ff_out += residual
        return ff_out, new_xl_memories



#### **MemorizingTransformer Class**
This is the main model class, which builds the entire Transformer network.

- **Initialization**:
  - It initializes embedding layers, KNN, relative position encoders, and multiple layers of attention blocks. The KNN is used to store and retrieve memories throughout the model's processing.

- **Forward Pass**:
  - The input sequence `x` is passed through multiple layers of attention and feedforward blocks. Each layer uses either relative position encoding or KNN-based attention, depending on the layer index.
  
- **Memory Management**:
  - The model maintains "XL memories," which are propagated through the layers. This allows the model to retain information from previous sequences and incorporate it into the current computation.
  
- **Loss Calculation**:
  - The output logits (predictions) are compared to the true labels using **cross-entropy loss**.

In [ ]:
class MemorizingTransformer(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        vocab_size,
        heads = 8,
        depth = 10,
        dropout = 0,
        head_dimension = 64,
        max_knn_memories = 32000,
        topk = 5,

    ):
        super().__init__()
        self.heads = heads
        self.embedding_dimension = embedding_dimension
        self.dropout = dropout
        self.depth = depth
        self.head_dimension = head_dimension
        self.max_knn_memories = max_knn_memories
        self.topk = topk
        self.rel_pos = RelativePosition(rp_scale = head_dimension** 0.5, heads = self.heads)
        self.rel_pos_knn = RelativePosition(rp_scale = head_dimension** 0.5, heads = self.heads)
        self.embedding_matrix = nn.Embedding(vocab_size, self.embedding_dimension)
        self.knn = KNN(head_dimension * heads, self.max_knn_memories)
        self.layers = nn.ModuleList([])

        for i in range(self.depth):

            if i == self.depth-2:
                layer_knn = self.knn
            else:
                layer_knn = None

            self.layers.append(Block(layer_knn))

        self.to_logits = nn.Sequential(
            nn.LayerNorm(self.embedding_dimension),
            nn.Linear(self.embedding_dimension, vocab_size)
        )

    def forward(
        self,
        x,
        relative_positions = None,
        xl_memories = None,
        labels = None,
    ):

        batch_size, sequence_length = x.shape[0], x.shape[1]

        # Position values
        rel_pos = self.rel_pos(sequence_length)
        rel_pos_knn = self.rel_pos_knn(sequence_length)

        if xl_memories is not None:
            xl_memories = xl_memories
        else:
            xl_memories = (None,) * self.depth

        # Iterator
        xl_memories_iter = iter(xl_memories)

        # Store the XL memories for each pass
        new_xl_memories = []

        # Embeddings
        x = self.embedding_matrix(x)

        for ind, block in enumerate(self.layers):

            if ind == self.depth-2:
                layer_rel_pos = rel_pos_knn
            else:
                layer_rel_pos = rel_pos

            x, xl_mem = block(x, next(xl_memories_iter), layer_rel_pos)

            if xl_mem is not None:
                new_xl_memories.append(xl_mem)

        logits = self.to_logits(x)

        loss = F.cross_entropy(rearrange(logits, 'b n c -> b c n'), labels)
        if len(new_xl_memories) > 0:
            return loss, new_xl_memories
        return loss

# Full Model

Your code represents an implementation of a transformer architecture with support for different attention mechanisms, such as XLAttention and KNNAttention, which incorporate memory mechanisms like KNN (K-Nearest Neighbors) and external memories for long-range dependencies. Here's a summary of key components:

1. **RelativePosition**: This class computes relative position embeddings, which help the model capture the relationship between positions in the input sequence. It supports a variety of configurations to determine the bucket indices based on the distance between sequence positions.

2. **KNN Class**: Implements a memory-based approach using K-Nearest Neighbors (KNN). It stores embeddings (keys and values) of previous sequences and retrieves the top-k nearest neighbors during attention, enabling the model to use external memory to enhance its ability to process long-range dependencies efficiently. This is achieved by combining the current sequence with previously stored embeddings.

3. **XLAttention**: A standard self-attention mechanism, enhanced with external memory (via `xl_memory`). This allows the model to perform attention over previously seen sequence tokens, supporting long-range dependencies without quadratic complexity growth.

4. **KNNAttention**: A modification of the self-attention mechanism that incorporates KNN-based retrieval of memory. The top-k nearest neighbors are retrieved for each query, allowing the model to access relevant past context dynamically. This approach enhances memory management for transformers handling long sequences, improving performance on tasks like document summarization.

5. **Block**: Each block consists of a combination of attention (either XLAttention or KNNAttention) and a feed-forward layer. It integrates residual connections, normalization, and dropout for regularization.

6. **MemorizingTransformer**: The main model class that integrates multiple transformer blocks. It uses both relative position encodings and either XL or KNN attention mechanisms. This class manages the processing of sequences, including retrieval of memory, updating memory, and predicting outputs using a final linear layer.

### Training Setup:
- The dataset being processed is the **arxiv-summarization** dataset, which has been pre-processed into chunks of text to fit within a transformer model's input constraints.
- A training loop is set up to use a dynamic batch size, learning rate, and gradient clipping to handle the model’s optimization.

This architecture is designed to improve long-term dependency handling in sequence models by utilizing memory mechanisms like KNN and external XL memory, which can be especially useful in tasks requiring processing of large documents or long sequences.

Let me know if you'd like further clarifications or have specific questions!

In [ ]:
class RelativePosition(nn.Module):
  def __init__(
      self,
      rp_scale,
      num_buckets = 32,
      rp_max_distance = 128,
      heads = 8
  ):
      super().__init__()
      self.scale = rp_scale
      self.num_buckets = num_buckets
      self.rp_max_distance = rp_max_distance
      self.relative_attention_embedding = nn.Embedding(num_buckets, heads)

  def relative_position_bucket(self, relative_position_matrix):
      n = -relative_position_matrix
      n = torch.max(n, torch.zeros_like(n))

      max_exact = self.num_buckets // 2

      is_small = n < max_exact
      val_if_large = max_exact + (torch.log(n.float() / max_exact) / math.log(self.rp_max_distance / max_exact) * (self.num_buckets - max_exact)).long()
      val_if_large = torch.min(val_if_large, torch.full_like(val_if_large, self.num_buckets - 1))

      return torch.where(is_small, n, val_if_large)

  def forward(self, sequence_length):

      sequence_pos = torch.arange(sequence_length, dtype=torch.long)
      context_pos = torch.arange(2 * sequence_length, dtype=torch.long)
      sequence_rel_pos = rearrange(sequence_pos, 'i -> i 1')
      context_rel_pos = rearrange(context_pos, 'j -> 1 j')
      rel_pos = context_rel_pos - sequence_rel_pos

      position_bucket_indices = self.relative_position_bucket(rel_pos)

      rp_values = self.relative_attention_embedding(position_bucket_indices)
      rp_values = rearrange(rp_values, 'i j h -> () h i j')
      return rp_values * self.scale



class KNN():
    def __init__(
        self,
        dim,
        max_memories,
        ):
        self.dim = dim
        self.max_memories = max_memories
        self.shape = (max_memories, 2, dim)
        self.db_offset = 0
        self.db_filepath = "./memory.memmap"
        self.db = np.memmap(self.db_filepath, mode = 'w+', dtype = np.float32, shape = self.shape)
        self.index = faiss.IndexFlatL2(dim)


    def add_to_db(self, new_data):
        new_data_len = new_data.shape[0]
        ids = (np.arange(new_data_len) + self.db_offset)
        self.db[ids] = new_data.detach().numpy()
        self.db_offset += new_data_len
        # Write to file
        self.db.flush()


    def search_and_retrieve(self, query_vecs, topk):
        query_vecs = query_vecs
        distances, indices = self.index.search(query_vecs, topk)
        kvs = self.db[indices]
        return kvs

    def add(self, new_data):
        # Input is b n 2 d, flatten to (b n) 2 d
        new_data = new_data.flatten(0,1)
        # Add to db
        self.add_to_db(new_data)
        # Only keys are used in knn index
        keys, vals = new_data.unbind(dim=-2)
        keys = keys.detach().numpy()
        # Add (b n) d tensors to index
        keys = np.ascontiguousarray(keys)
        # Add to index
        self.index.add(keys)

    def search(self, query_vecs, topk):
        # can override topk
        query_batch_size, query_seq_len = query_vecs.shape[0], query_vecs.shape[1]
        # Input is b n d, flatten to (b n) d
        query_vecs = query_vecs.flatten(0,1)
        kvs = self.search_and_retrieve(np.ascontiguousarray(query_vecs.detach().numpy()), topk)
        # kvs are (b n) k 2 d, unflatten to b n k 2 d
        kvs = torch.tensor(kvs)
        kvs = torch.unflatten(kvs, 0, (query_batch_size, query_seq_len))
        return kvs


    def clear(self):
        self.index.reset()
        self.db[:] = 0
        self.db_offset = 0


class XLAttention(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        heads = 8,
        head_dimension = 64,
        dropout = 0.,
    ):
        super().__init__()
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        self.scale = head_dimension ** -0.5

        self.query_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.output_matrix = nn.Linear(self.heads * head_dimension, embedding_dimension)

    def forward(
        self,
        x, # batch_size, sequence_length, embedding_dimension
        relative_positions = None,
        xl_memory = None
    ):

        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        queries = queries * self.scale

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim = -2) # assume stacked
            keys = torch.cat((k_xl, keys), dim = -2) # prepend XL memory
            values = torch.cat((v_xl, values), dim = -2) # prepend XL memory
            xl_sequence_length = k_xl.shape[1]

        queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
        keys    = rearrange(keys, 'b t (h d) -> b h t d', h = self.heads)
        qk      = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk

        qk = qk * self.scale

        mask = torch.ones((i,j), dtype = torch.bool).triu(j-i+1)
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)
        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk@values
        qkv = rearrange(qkv, 'b h t d -> b t (h d)')

        out = self.output_matrix(qkv)

        # new XL memories

        keys = rearrange(keys, 'b h t d -> b t (h d)', h = self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2) # (batch, sequence_len, 2, dimension)


        if xl_memory is not None:
            xl_memories, current_input = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
            kv_to_add_xl = current_input
        else:
            kv_to_add_xl = kv_memories

        return out, kv_to_add_xl



class KNNAttention(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        knn,
        heads = 8,
        head_dimension = 64,
        topk_retrieved_memories = 3,
        dropout = 0.
    ):
        super().__init__()
        self.heads = heads
        self.scale = head_dimension ** -0.5
        self.dropout = nn.Dropout(dropout)

        self.query_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.output_matrix = nn.Linear(heads * head_dimension, embedding_dimension)

        self.gate_bias = nn.Parameter(torch.randn(self.heads, 1, 1))
        self.topk_retrieved_memories = topk_retrieved_memories
        self.knn = knn

    def forward(
        self,
        x, # batch_size, sequence_length, embedding_dimension
        relative_positions = None,
        xl_memory = None
    ):
        batch_size, sequence_length = x.shape[:2]
        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        queries = F.normalize(queries, dim=-1)
        keys = F.normalize(keys, dim=-1)

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim = -2) # unstack
            keys = torch.cat((k_xl, keys), dim = -2) # prepend XL memory
            values = torch.cat((v_xl, values), dim = -2) # prepend XL memory
            xl_sequence_length = k_xl.shape[1]

        ### LOCAL ATTENTION

        queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
        keys    = rearrange(keys, 'b t (h d) -> b h t d', h = self.heads)
        qk      = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk

        qk = qk * self.scale

        mask = torch.ones((i,j), dtype = torch.bool).triu(j-i+1)
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)

        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk@values

        ### KNN ATTENTION

        # If there are knn memories (we're not on the first segment) then perform knn attention
        if self.knn.index.ntotal > 0:
            # Convert queries to search form
            queries = rearrange(queries, 'b h t d -> b t (h d)')
            mem_kv = self.knn.search(queries, topk = self.topk_retrieved_memories) # returns b t k 2 d
            mem_k, mem_v = mem_kv.unbind(dim = -2)
            mem_k = rearrange(mem_k, 'b t k (h d) -> b h t k d', h=self.heads)
            mem_v = rearrange(mem_v, 'b t k (h d) -> b h t k d', h=self.heads)

            # Convert queries to attention form
            queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
            mem_qk = einsum(queries, mem_k, 'b h t d, b h t k d -> b h t k')
            mem_qk = mem_qk * self.scale

            mem_qk = F.softmax(mem_qk, dim=-1)
            mem_qk = self.dropout(mem_qk)
            mem_qkv = einsum(mem_qk, mem_v, 'b h t k, b h t k d -> b h t d')

            # Combined attentions

            combined_qkv = mem_qkv * self.gate_bias + qkv * (1 - self.gate_bias)
            combined_qkv = rearrange(combined_qkv, 'b h t d -> b t (h d)')
            out = self.output_matrix(combined_qkv)

        else:
            qkv = rearrange(qkv, 'b h t d -> b t (h d)')
            out = self.output_matrix(qkv)

        # New XL memories
        keys = rearrange(keys, 'b h t d -> b t (h d)', h = self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2) # (batch, sequence_len, 2, dimension)

        if xl_memory is not None:
            # if we're on a middle/end segment of a document (there are previous XL memories)
            xl_memories, current_kv = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
        else:
            # if we're at the first segment
            current_kv = kv_memories

        self.knn.add(current_kv)

        return out, current_kv


class Block(nn.Module):
    def __init__(self, embedding_dimension, attention_type, dropout=0.):
        super().__init__()
        self.attention = attention_type
        self.dim = embedding_dimension

        self.ff_block = nn.Sequential(
            nn.LayerNorm(self.dim),
            nn.Linear(self.dim, self.dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(self.dim * 4, self.dim))

    def forward(self, x, xl_memories, rel_pos):
        residual = x
        norm = nn.LayerNorm(self.dim)
        attn_out = norm(x)
        attn_out, new_xl_memories = self.attention(attn_out, relative_positions=rel_pos, xl_memory=xl_memories)
        attn_out += residual

        residual = attn_out
        ff_out = self.ff_block(attn_out)
        ff_out += residual
        return ff_out, new_xl_memories


class MemorizingTransformer(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        vocab_size,
        max_knn_memories = 81920,
        heads = 8,
        depth = 10,
        dropout = 0,
        head_dimension = 64,
        topk = 5,

    ):
        super().__init__()
        self.heads = heads
        self.embedding_dimension = embedding_dimension
        self.dropout = dropout
        self.depth = depth
        self.head_dimension = head_dimension
        self.max_knn_memories = max_knn_memories
        self.topk = topk

        ###########
        self.rel_pos = RelativePosition(rp_scale = head_dimension** 0.5,
                                        heads = self.heads)
        self.rel_pos_knn = RelativePosition(rp_scale = head_dimension** 0.5,
                                        heads = self.heads)
        self.embedding_matrix = nn.Embedding(vocab_size, self.embedding_dimension)

        self.knn = KNN(head_dimension * heads, self.max_knn_memories)



        self.layers = nn.ModuleList([])
        for i in range(self.depth):

            if i == self.depth-2:
                attention_type = KNNAttention(self.embedding_dimension,
                            self.knn,
                            heads = self.heads,
                            head_dimension = self.head_dimension,
                            dropout = self.dropout)
            else:
                attention_type = XLAttention(self.embedding_dimension,
                            heads = self.heads,
                            head_dimension = self.head_dimension,
                            dropout = self.dropout)

            self.layers.append(Block(self.embedding_dimension, attention_type))

        self.to_logits = nn.Sequential(
            nn.LayerNorm(self.embedding_dimension),
            nn.Linear(self.embedding_dimension, vocab_size)
        )


    def forward(
        self,
        x,
        relative_positions = None,
        xl_memories = None,
        labels = None,
    ):

        batch_size, sequence_length = x.shape[0], x.shape[1]

        # Position values
        rel_pos = self.rel_pos(sequence_length)
        rel_pos_knn = self.rel_pos_knn(sequence_length)

        # If no XL memories (start of a sequence) then None type for each layer.
        # There is one set of XL memories for each layer
        # xl_memories = default(xl_memories, (None,) * self.num_xl_memory_layers)
        if xl_memories is not None:
            xl_memories = xl_memories
        else:
            xl_memories = (None,) * self.depth

        # Iterator
        xl_memories_iter = iter(xl_memories)

        # Embeddings
        x = self.embedding_matrix(x)

        # Store the XL memories for each pass
        new_xl_memories = []

        for ind, block in enumerate(self.layers):

            if i == self.depth-2:
                layer_rel_pos = rel_pos_knn
            else:
                layer_rel_pos = rel_pos

            x, xl_mem = block(x, next(xl_memories_iter), layer_rel_pos)

            if xl_mem is not None:
                ############
                new_xl_memories.append(xl_mem.detach())



        logits = self.to_logits(x)

        # Training
        loss = F.cross_entropy(rearrange(logits, 'b n c -> b c n'), labels)
        if len(new_xl_memories) > 0:
            return loss, new_xl_memories
        return loss

# Training loop

Your code outlines a training loop for a custom transformer model, `MemorizingTransformer`, with enhancements for memory management using both KNN-based memory and XL memory mechanisms. Here's a breakdown of key aspects and some points to consider:

### Key Elements of Your Training Loop:

1. **Model Setup**:
   - The model is initialized with `embedding_dimension = 128` and `vocab_size = 128`, implying a relatively small embedding space. The `max_knn_memories` parameter limits the amount of memory retained by the KNN mechanism.
   - The optimizer used is Adam with a learning rate defined by `LEARNING_RATE`.

2. **Training Loop**:
   - The outer loop runs for 200 iterations, and the training loss is calculated for each batch. You clear both the XL and KNN memories at the beginning of each iteration, allowing the model to learn afresh.
   - **Memory Clearing**: The use of `xl_memories = None` and `model.knn.clear()` ensures that no old memory persists between training iterations, which can help avoid excessive memory usage and stale context.
   - **Sequence Segmentation**: The input sequences (`seq`) and labels (`labels`) are chunked into smaller segments for training. This is beneficial when working with long sequences, as it reduces memory overhead and allows the model to focus on smaller portions of the sequence at a time.
   
3. **Loss Calculation**:
   - The loss is computed for each segment and averaged. The backward pass is done for each segment, and gradients are clipped using `torch.nn.utils.clip_grad_norm_` to avoid gradient explosion.

4. **Validation**:
   - Every few steps (`VALIDATE_EVERY`), the model switches to evaluation mode (`model.eval()`). It then processes validation data in a similar manner as the training loop but without gradient computation (`torch.no_grad()`).

### Potential Improvements or Considerations:

- **Memory Management**: If you're using external memory such as KNN or XL memories, the model's memory management should be carefully handled. This means ensuring that memories are cleared only when appropriate, and potentially saving key states between epochs or segments if needed for consistency across batches.
- **Efficient Data Loading**: If your data is large, consider optimizing the way data is loaded using `DataLoader` or parallel processing to ensure you're not bottlenecked by disk I/O.
- **Learning Rate and Grad Clipping**: The learning rate and gradient clipping threshold (`MAX_GRAD_CLIP_NORM`) should be tuned to the specific problem to prevent overfitting or slow convergence.

This setup appears to be designed for handling sequences with long-range dependencies, particularly where standard attention mechanisms may struggle due to memory constraints.

In [ ]:
SEGMENTS = 10
SEQUENCE_LENGTH = 512
CHUNK_SIZE = (SEGMENTS * SEQUENCE_LENGTH) + 1 #### we need +1
BATCH_SIZE = 8
LEARNING_RATE = 2e-4
MAX_GRAD_CLIP_NORM = 0.5
VALIDATE_EVERY = 100
MAX_KNN_MEMORIES = BATCH_SIZE * 1 * SEQUENCE_LENGTH * SEGMENTS


dataset = datasets.load_dataset("ccdv/arxiv-summarization", split='train', streaming=True)
raw_dataset = list(dataset.take(3500))

raw_articles = [x['article'] for x in raw_dataset]
raw_articles = [x for x in raw_articles if len(x) > CHUNK_SIZE]
converted = [np.fromstring(doc, dtype=np.uint8) for doc in raw_articles]

def clip_article(doc, chunk_size):
    remainder = len(doc) % chunk_size
    return doc[:-remainder]

clipped = [clip_article(doc, CHUNK_SIZE) for doc in converted]
chunked = np.array([doc.reshape(-1, CHUNK_SIZE) for doc in clipped])
processed_data = torch.tensor(np.concatenate(chunked), dtype=torch.long)
processed_data.shape

eighty_split = int(processed_data.shape[0] * .8)
ninety_split = int(processed_data.shape[0] * .9)
train_loader = iter(DataLoader(processed_data[:eighty_split], batch_size = BATCH_SIZE, shuffle = True))
val_loader = iter(DataLoader(processed_data[eighty_split:ninety_split], batch_size = BATCH_SIZE, shuffle = True))
test_loader = iter(DataLoader(processed_data[ninety_split:], batch_size = BATCH_SIZE, shuffle = True))

<ipython-input-9-335de34fcf28>:16: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  converted = [np.fromstring(doc, dtype=np.uint8) for doc in raw_articles]
<ipython-input-9-335de34fcf28>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  chunked = np.array([doc.reshape(-1, CHUNK_SIZE) for doc in clipped])


In [ ]:
model = MemorizingTransformer(embedding_dimension = 128,
                              vocab_size = 128,
                              max_knn_memories = MAX_KNN_MEMORIES)

optim = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
model.train()


for i in tqdm.tqdm(range(200), mininterval = 10., desc = 'training'):

    model.train()
    train_loss = 0.
    #########
    # Clear XL memories
    xl_memories = None
    #########
    # Clear KNN memory
    model.knn.clear()

    data = next(train_loader)
    seq, labels = data[:, :-1], data[:, 1:]


    # Each pass will be (BATCH_SIZE * SEGMENTS) iterations
    for seq_segment, labels_segment in zip(seq.chunk(SEGMENTS, dim = -1), labels.chunk(SEGMENTS, dim = -1)):

        loss, xl_memories = model(
            seq_segment,
            labels = labels_segment,
            xl_memories = xl_memories
        )

        train_loss += loss.item() / SEGMENTS
        (loss / SEGMENTS).backward()
        print ("segment complete")


    print(f'training loss: {train_loss}')
    torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_CLIP_NORM)
    optim.step()
    optim.zero_grad()


    if not (i % VALIDATE_EVERY):
        model.eval()

        valid_data = next(val_loader)
        valid_loss = 0.

        with torch.no_grad():
            xl_memories = None
            model.knn.clear()
            seq, labels = data[:, :-1], data[:, 1:]

            for seq_segment, labels_segment in zip(seq.chunk(SEGMENTS, dim = -1), labels.chunk(SEGMENTS, dim = -1)):

                loss, xl_memories = model(
                    seq_segment,
                    labels = labels_segment,
                    xl_memories = xl_memories
                )

                valid_loss += loss.item() / SEGMENTS

        print(f'valid loss: {valid_loss}')

training:   0%|          | 0/200 [00:00<?, ?it/s]

segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
training loss: 4.96113452911377


training:   0%|          | 1/200 [02:15<7:31:01, 135.99s/it]

valid loss: 4.396467971801758
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete


training:   1%|          | 2/200 [03:45<5:59:15, 108.87s/it]

segment complete
training loss: 4.425439643859864
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete
segment complete


training:   1%|          | 2/200 [05:13<8:37:28, 156.81s/it]


KeyboardInterrupt: ignored

# End to End GPU

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import math
import os
import random
import tqdm
import gzip
import time

!pip install einops
from einops import rearrange, repeat, pack, unpack, einsum
from einops.layers.torch import Rearrange


from functools import partial, wraps
from contextlib import contextmanager, ExitStack
from pathlib import Path
from filelock import FileLock
import pickle

import transformers
from transformers import AutoTokenizer

!pip install faiss-gpu
import faiss

!pip install datasets
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [ ]:
# can check our GPU

In [ ]:
!nvidia-smi

Thu Dec 21 17:48:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The code provided outlines a large and complex deep learning model built using PyTorch. The model seems to be a variant of a transformer model with memory and attention mechanisms, specifically integrating a KNN-based (k-nearest neighbors) attention mechanism. Here's a breakdown of the key components:

### 1. **Relative Positioning** (`RelativePosition` class):
   - This class calculates relative positions of sequence elements in the input data. The relative position embeddings help the model learn dependencies between elements at various distances.
   - It utilizes a technique that buckets positions into a predefined number of ranges (`num_buckets`) to handle distances more effectively.
   - This is used in the main attention mechanism to improve performance over simple absolute position embeddings.

### 2. **KNN Memory** (`KNN` class):
   - This component provides a memory mechanism that stores and retrieves embeddings (or representations) of sequences to enhance performance.
   - It uses FAISS, a library for efficient similarity search, to perform nearest neighbor search and add new data to the memory (a form of external memory augmentation for transformers).

### 3. **Attention Mechanisms**:
   - The **XLAttention** class implements a standard transformer-style attention mechanism with the addition of extended memory (XL memory) for longer sequences.
   - The **KNNAttention** class adds the KNN-based attention mechanism that allows the model to retrieve relevant past memories during the attention calculation, improving the model’s ability to handle longer contexts and provide more relevant attention.

### 4. **Transformer Block** (`Block` class):
   - Each block contains a layer of attention followed by a feed-forward network. The attention type in each block could either be the standard XLAttention or the KNNAttention, depending on the depth of the block.
   - The `forward` method computes both the attention output and the updated memory.

### 5. **Main Model** (`MemorizingTransformer` class):
   - The `MemorizingTransformer` class is the overall transformer model that integrates these components.
   - It defines multiple layers of attention blocks (either XL or KNN-based) and manages the storage and retrieval of XL memory across segments.
   - The model is designed to handle long sequences and includes an optimized method for computing attention over long-term memory using KNN retrieval.
   - The final output is passed through a linear layer to produce logits, and a cross-entropy loss is computed for training.

### 6. **Data Preparation**:
   - The code uses the `ccdv/arxiv-summarization` dataset to train the model. It splits the dataset into chunks, ensuring that each chunk size matches the model's required input size (CHUNK_SIZE).
   - The data is then processed into batches, ready to be fed into the model for training.

### 7. **Training Setup**:
   - The model is set to be trained with a learning rate of `2e-4` and gradient clipping at a norm of `0.5`.
   - Training, validation, and test data loaders are created from the processed dataset.

### High-Level Summary:
This model is a custom transformer with both XL memory and KNN-based attention, designed to efficiently handle long sequences of text by leveraging external memory and relevant context from previous inputs. It processes large chunks of text data, integrates relative position embeddings, and dynamically retrieves memory during attention calculations. The training setup uses typical cross-entropy loss and data batching techniques suitable for text generation or summarization tasks.

In [ ]:
class RelativePosition(nn.Module):
  def __init__(
      self,
      rp_scale,
      num_buckets = 32,
      rp_max_distance = 128,
      heads = 8
  ):
      super().__init__()
      self.scale = rp_scale
      self.num_buckets = num_buckets
      self.rp_max_distance = rp_max_distance
      self.relative_attention_embedding = nn.Embedding(num_buckets, heads)

  def relative_position_bucket(self, relative_position_matrix):
      n = -relative_position_matrix
      n = torch.max(n, torch.zeros_like(n))

      max_exact = self.num_buckets // 2

      is_small = n < max_exact
      val_if_large = max_exact + (torch.log(n.float() / max_exact) / math.log(self.rp_max_distance / max_exact) * (self.num_buckets - max_exact)).long()
      val_if_large = torch.min(val_if_large, torch.full_like(val_if_large, self.num_buckets - 1))

      return torch.where(is_small, n, val_if_large)

  def forward(self, sequence_length, device):

      sequence_pos = torch.arange(sequence_length, dtype=torch.long, device=device) ##########
      context_pos = torch.arange(2 * sequence_length, dtype=torch.long, device=device) ###########
      sequence_rel_pos = rearrange(sequence_pos, 'i -> i 1')
      context_rel_pos = rearrange(context_pos, 'j -> 1 j')
      rel_pos = context_rel_pos - sequence_rel_pos

      position_bucket_indices = self.relative_position_bucket(rel_pos)

      rp_values = self.relative_attention_embedding(position_bucket_indices)
      rp_values = rearrange(rp_values, 'i j h -> () h i j')
      return rp_values * self.scale



class KNN():
    def __init__(
        self,
        dim,
        max_memories,
        ):
        self.dim = dim
        self.max_memories = max_memories
        self.shape = (max_memories, 2, dim)
        self.db_offset = 0
        self.db_filepath = "./memory.memmap"
        self.db = np.memmap(self.db_filepath, mode = 'w+', dtype = np.float32, shape = self.shape)
        self.index = faiss.IndexFlatL2(dim)


    def add_to_db(self, new_data):
        new_data_len = new_data.shape[0]
        ids = (np.arange(new_data_len) + self.db_offset)
        self.db[ids] = new_data.detach().cpu().numpy() #######
        self.db_offset += new_data_len
        # Write to file
        self.db.flush()


    def search_and_retrieve(self, query_vecs, topk):
        query_vecs = query_vecs
        distances, indices = self.index.search(query_vecs, topk)
        kvs = self.db[indices]
        return kvs

    def add(self, new_data):
        # Input is b n 2 d, flatten to (b n) 2 d
        new_data = new_data.flatten(0,1)
        # Add to db
        self.add_to_db(new_data)
        # Only keys are used in knn index
        keys, vals = new_data.unbind(dim=-2)
        keys = keys.detach().cpu().numpy() ######
        # Add (b n) d tensors to index
        keys = np.ascontiguousarray(keys)
        # Add to index
        self.index.add(keys)

    def search(self, query_vecs, topk):
        query_batch_size, query_seq_len = query_vecs.shape[0], query_vecs.shape[1]
        device = query_vecs.device ######
        # Input is b n d, flatten to (b n) d
        query_vecs = query_vecs.flatten(0,1)
        kvs = self.search_and_retrieve(np.ascontiguousarray(query_vecs.detach().cpu().numpy()), topk) ######
        # kvs are (b n) k 2 d, unflatten to b n k 2 d
        kvs = torch.tensor(kvs)
        kvs = torch.unflatten(kvs, 0, (query_batch_size, query_seq_len))
        return kvs.to(device)

    def clear(self):
        self.index.reset()
        self.db[:] = 0
        self.db_offset = 0


class XLAttention(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        heads = 8,
        head_dimension = 64,
        dropout = 0.,
    ):
        super().__init__()
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        self.scale = head_dimension ** -0.5

        self.query_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.output_matrix = nn.Linear(self.heads * head_dimension, embedding_dimension)

    def forward(
        self,
        x, # batch_size, sequence_length, embedding_dimension
        relative_positions = None,
        xl_memory = None
    ):

        device = x.device ##########
        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        queries = queries * self.scale

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim = -2) # assume stacked
            keys = torch.cat((k_xl, keys), dim = -2) # prepend XL memory
            values = torch.cat((v_xl, values), dim = -2) # prepend XL memory
            xl_sequence_length = k_xl.shape[1]

        queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
        keys    = rearrange(keys, 'b t (h d) -> b h t d', h = self.heads)
        qk      = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk

        qk = qk * self.scale

        mask = torch.ones((i,j), dtype = torch.bool, device=device).triu(j-i+1) ########
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)
        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk@values
        qkv = rearrange(qkv, 'b h t d -> b t (h d)')

        out = self.output_matrix(qkv)

        # new XL memories

        keys = rearrange(keys, 'b h t d -> b t (h d)', h = self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2) # (batch, sequence_len, 2, dimension)


        if xl_memory is not None:
            xl_memories, current_input = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
            kv_to_add_xl = current_input
        else:
            kv_to_add_xl = kv_memories

        return out, kv_to_add_xl



class KNNAttention(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        knn,
        heads = 8,
        head_dimension = 64,
        topk_retrieved_memories = 3,
        dropout = 0.
    ):
        super().__init__()
        self.heads = heads
        self.scale = head_dimension ** -0.5
        self.dropout = nn.Dropout(dropout)

        self.query_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.output_matrix = nn.Linear(heads * head_dimension, embedding_dimension)

        self.gate_bias = nn.Parameter(torch.randn(self.heads, 1, 1))
        self.topk_retrieved_memories = topk_retrieved_memories
        self.knn = knn

    def forward(
        self,
        x, # batch_size, sequence_length, embedding_dimension
        relative_positions = None,
        xl_memory = None
    ):

        device = x.device ########
        batch_size, sequence_length = x.shape[:2]
        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        queries = F.normalize(queries, dim=-1)
        keys = F.normalize(keys, dim=-1)

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim = -2) # unstack
            keys = torch.cat((k_xl, keys), dim = -2) # prepend XL memory
            values = torch.cat((v_xl, values), dim = -2) # prepend XL memory
            xl_sequence_length = k_xl.shape[1]

        ### LOCAL ATTENTION

        queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
        keys    = rearrange(keys, 'b t (h d) -> b h t d', h = self.heads)
        qk      = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk

        qk = qk * self.scale

        mask = torch.ones((i,j), dtype = torch.bool, device=device).triu(j-i+1) ########
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)

        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk@values

        ### KNN ATTENTION

        # If there are knn memories (we're not on the first segment) then perform knn attention
        if self.knn.index.ntotal > 0:
            t1 = time.time()
            print ("Begin KNN operations")
            # Convert queries to search form
            queries = rearrange(queries, 'b h t d -> b t (h d)')
            mem_kv = self.knn.search(queries, topk = self.topk_retrieved_memories) # returns b t k 2 d
            mem_k, mem_v = mem_kv.unbind(dim = -2)
            mem_k = rearrange(mem_k, 'b t k (h d) -> b h t k d', h=self.heads)
            mem_v = rearrange(mem_v, 'b t k (h d) -> b h t k d', h=self.heads)

            # Convert queries to attention form
            queries = rearrange(queries, 'b t (h d) -> b h t d', h = self.heads)
            mem_qk = einsum(queries, mem_k, 'b h t d, b h t k d -> b h t k')
            mem_qk = mem_qk * self.scale

            mem_qk = F.softmax(mem_qk, dim=-1)
            mem_qk = self.dropout(mem_qk)
            mem_qkv = einsum(mem_qk, mem_v, 'b h t k, b h t k d -> b h t d')

            # Combined attentions

            combined_qkv = mem_qkv * self.gate_bias + qkv * (1 - self.gate_bias)
            combined_qkv = rearrange(combined_qkv, 'b h t d -> b t (h d)')
            out = self.output_matrix(combined_qkv)
            t2 = time.time()
            print ("End KNN operations, time taken:", t2-t1)

        else:
            qkv = rearrange(qkv, 'b h t d -> b t (h d)')
            out = self.output_matrix(qkv)

        # New XL memories
        keys = rearrange(keys, 'b h t d -> b t (h d)', h = self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2) # (batch, sequence_len, 2, dimension)

        if xl_memory is not None:
            # if we're on a middle/end segment of a document (there are previous XL memories)
            xl_memories, current_kv = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
        else:
            # if we're at the first segment
            current_kv = kv_memories

        self.knn.add(current_kv)

        return out, current_kv


class Block(nn.Module):
    def __init__(self, embedding_dimension, attention_type, dropout=0.):
        super().__init__()
        self.attention = attention_type
        self.dim = embedding_dimension
        self.norm = nn.LayerNorm(self.dim)

        self.ff_block = nn.Sequential(
            nn.LayerNorm(self.dim),
            nn.Linear(self.dim, self.dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(self.dim * 4, self.dim))

    def forward(self, x, xl_memories, rel_pos):
        residual = x
        #norm = nn.LayerNorm(self.dim) #########
        attn_out = self.norm(x)
        attn_out, new_xl_memories = self.attention(attn_out, relative_positions=rel_pos, xl_memory=xl_memories)
        attn_out += residual

        residual = attn_out
        ff_out = self.ff_block(attn_out)
        ff_out += residual
        return ff_out, new_xl_memories


class MemorizingTransformer(nn.Module):
    def __init__(
        self,
        embedding_dimension,
        vocab_size,
        max_knn_memories = 81920,
        heads = 8,
        depth = 10,
        dropout = 0,
        head_dimension = 64,
        topk = 5,

    ):
        super().__init__()
        self.heads = heads
        self.embedding_dimension = embedding_dimension
        self.dropout = dropout
        self.depth = depth
        self.head_dimension = head_dimension
        self.max_knn_memories = max_knn_memories
        self.topk = topk

        ###########
        self.rel_pos = RelativePosition(rp_scale = head_dimension** 0.5,
                                        heads = self.heads)
        self.rel_pos_knn = RelativePosition(rp_scale = head_dimension** 0.5,
                                        heads = self.heads)
        self.embedding_matrix = nn.Embedding(vocab_size, self.embedding_dimension)

        self.knn = KNN(head_dimension * heads, self.max_knn_memories)



        self.layers = nn.ModuleList([])
        for i in range(self.depth):

            if i == self.depth-2:
                attention_type = KNNAttention(self.embedding_dimension,
                            self.knn,
                            heads = self.heads,
                            head_dimension = self.head_dimension,
                            dropout = self.dropout)
            else:
                attention_type = XLAttention(self.embedding_dimension,
                            heads = self.heads,
                            head_dimension = self.head_dimension,
                            dropout = self.dropout)

            self.layers.append(Block(self.embedding_dimension, attention_type))

        self.to_logits = nn.Sequential(
            nn.LayerNorm(self.embedding_dimension),
            nn.Linear(self.embedding_dimension, vocab_size)
        )


    def forward(
        self,
        x,
        relative_positions = None,
        xl_memories = None,
        labels = None,
    ):

        device = x.device ########
        batch_size, sequence_length = x.shape[0], x.shape[1]

        # Position values
        rel_pos = self.rel_pos(sequence_length, device=device) ########
        rel_pos_knn = self.rel_pos_knn(sequence_length, device=device) ########

        # If no XL memories (start of a sequence) then None type for each layer.
        # There is one set of XL memories for each layer
        # xl_memories = default(xl_memories, (None,) * self.num_xl_memory_layers)
        if xl_memories is not None:
            xl_memories = xl_memories
        else:
            xl_memories = (None,) * self.depth

        # Iterator
        xl_memories_iter = iter(xl_memories)

        # Embeddings
        x = self.embedding_matrix(x)

        # Store the XL memories for each pass
        new_xl_memories = []

        for ind, block in enumerate(self.layers):

            if i == self.depth-2:
                layer_rel_pos = rel_pos_knn
            else:
                layer_rel_pos = rel_pos

            x, xl_mem = block(x, next(xl_memories_iter), layer_rel_pos)

            if xl_mem is not None:
                ############
                new_xl_memories.append(xl_mem.detach())



        logits = self.to_logits(x)

        # Training
        loss = F.cross_entropy(rearrange(logits, 'b n c -> b c n'), labels)
        if len(new_xl_memories) > 0:
            return loss, new_xl_memories
        return loss

The code you've shared preprocesses the **Arxiv Summarization Dataset** for use in a deep learning model. Here's a breakdown of the key steps involved in the preprocessing:

1. **Dataset Loading**:
   - The dataset `ccdv/arxiv-summarization` is loaded using `datasets.load_dataset()`, specifically the 'train' split, which is streamed to avoid loading the entire dataset into memory at once.
   - The `take(3500)` function fetches the first 3500 articles from the dataset.

2. **Filtering Articles**:
   - The articles are filtered to ensure that they are large enough to be split into chunks. This is done by checking if their length exceeds the `CHUNK_SIZE`.
   - The articles are then converted to a `numpy` array of `uint8` values.

3. **Chunking**:
   - The function `clip_article()` ensures that each article’s length is a multiple of `CHUNK_SIZE`, trimming the last part if needed.
   - After clipping, the articles are reshaped into chunks, each of size `CHUNK_SIZE` (which is determined by multiplying the number of segments by the sequence length).
   - This results in a 2D array of shape `(num_articles, num_chunks)`.

4. **Data Preparation**:
   - The articles are concatenated into a single tensor, `processed_data`, which is then split into training, validation, and test sets.
   - The dataset is split in an 80-10-10 ratio (train, validation, test), with each split fed into separate `DataLoader` instances for batching.

5. **Hyperparameters**:
   - `BATCH_SIZE` is set to 8, meaning 8 chunks of data will be processed at a time.
   - `SEQUENCE_LENGTH` and `SEGMENTS` define the size of each chunk, with the total `CHUNK_SIZE` calculated as `(SEGMENTS * SEQUENCE_LENGTH) + 1`.
   - The code also specifies other hyperparameters like `LEARNING_RATE`, `MAX_GRAD_CLIP_NORM`, and the maximum number of KNN memories (`MAX_KNN_MEMORIES`).

This data preparation process is essential for training the transformer model efficiently, particularly when dealing with large datasets such as the Arxiv Summarization Dataset. By chunking articles and splitting the dataset into manageable batches, the model can be trained on sequences of meaningful length without overwhelming memory limitations.

In [ ]:
SEGMENTS = 10
SEQUENCE_LENGTH = 512
CHUNK_SIZE = (SEGMENTS * SEQUENCE_LENGTH) + 1
BATCH_SIZE = 8
LEARNING_RATE = 2e-4
MAX_GRAD_CLIP_NORM = 0.5
VALIDATE_EVERY = 100
MAX_KNN_MEMORIES = BATCH_SIZE * 1 * SEQUENCE_LENGTH * SEGMENTS


dataset = datasets.load_dataset("ccdv/arxiv-summarization", split='train', streaming=True)
raw_dataset = list(dataset.take(3500))

raw_articles = [x['article'] for x in raw_dataset]
raw_articles = [x for x in raw_articles if len(x) > CHUNK_SIZE]
converted = [np.fromstring(doc, dtype=np.uint8) for doc in raw_articles]

def clip_article(doc, chunk_size):
    remainder = len(doc) % chunk_size
    return doc[:-remainder]

clipped = [clip_article(doc, CHUNK_SIZE) for doc in converted]


chunked = np.array([doc.reshape(-1, CHUNK_SIZE) for doc in clipped])

processed_data = torch.tensor(np.concatenate(chunked), dtype=torch.long)
processed_data.shape
eighty_split = int(processed_data.shape[0] * .8)
ninety_split = int(processed_data.shape[0] * .9)
train_loader = iter(DataLoader(processed_data[:eighty_split], batch_size = BATCH_SIZE, shuffle = True))
val_loader = iter(DataLoader(processed_data[eighty_split:ninety_split], batch_size = BATCH_SIZE, shuffle = True))
test_loader = iter(DataLoader(processed_data[ninety_split:], batch_size = BATCH_SIZE, shuffle = True))

<ipython-input-5-e456fa9b9fe6>:16: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  converted = [np.fromstring(doc, dtype=np.uint8) for doc in raw_articles]
<ipython-input-5-e456fa9b9fe6>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  chunked = np.array([doc.reshape(-1, CHUNK_SIZE) for doc in clipped])


In the provided code, the **training loop** for the **Memorizing Transformer** model is implemented using the `torch` library. Here's a breakdown of what's happening in each part:

### Key Steps in the Training Loop:

1. **Model Initialization**:
   - The model is instantiated with `MemorizingTransformer`, using an `embedding_dimension` of 128 and `vocab_size` of 128. This means the transformer operates in an embedding space of 128-dimensional vectors, and its vocabulary size (number of unique tokens) is also 128.
   - `MAX_KNN_MEMORIES` is passed as a hyperparameter for managing the KNN memory size in the model.

2. **Optimizer Setup**:
   - The Adam optimizer is used to optimize the model’s parameters with a learning rate of `LEARNING_RATE` (set to `2e-4`).
   - The optimizer step is executed after each training iteration.

3. **Training Loop**:
   - The outer loop runs for 200 iterations (`range(200)`). During each iteration:
     - **Training Mode**: The model is set to training mode with `model.train()`.
     - The training loss is initialized and stored in `train_loss`. The KNN memory is also cleared at the start of each iteration (`model.knn.clear()`).
   
4. **Processing a Batch**:
   - A batch of data is fetched from the training data loader and moved to the appropriate device (e.g., GPU) using `data.to(device)`.
   - The batch is split into `seq` (input sequences) and `labels` (target sequences) where `seq` is everything except the last token, and `labels` is everything except the first token (for next-token prediction).

5. **Chunking and Model Forward Pass**:
   - The sequences (`seq`) and labels (`labels`) are split into smaller chunks using `.chunk(SEGMENTS, dim=-1)`. Each chunk represents one segment of the input sequence, where `SEGMENTS` defines how many chunks will be processed per document.
   - In each chunk, the model computes the loss, and `backward()` is called to accumulate gradients.
   - Gradients are clipped with `torch.nn.utils.clip_grad_norm_()` to avoid gradient explosion, a common issue with deep learning models like transformers.

6. **Validation**:
   - Every `VALIDATE_EVERY` iterations, the model enters evaluation mode (`model.eval()`), and validation data is used to compute the validation loss.
   - Validation loss is computed similarly to the training loss, but with `torch.no_grad()` to avoid computing gradients (since no parameter updates are needed during validation).
   - After processing the validation batch, the validation loss is printed out.

### Things to Note:
- **KNN Memory Management**:
   - The model makes use of KNN memory, where `model.knn.clear()` clears the memory between training iterations. This likely helps the model in retrieving past memories to influence future predictions, an important feature of memory-augmented neural networks.
   
- **Handling Segments**:
   - The chunking mechanism (`seq.chunk(SEGMENTS, dim=-1)`) splits sequences into smaller segments. This allows the model to handle long sequences efficiently without exceeding memory limits.

- **Loss Backpropagation**:
   - The loss is calculated for each segment (`loss / SEGMENTS`) and then backpropagated using `loss.backward()`. This ensures that the gradients are computed for each segment and accumulated before applying the optimizer step.

### Potential Improvements:
- **Gradient Accumulation**: If memory issues arise, gradient accumulation can be employed, where gradients are accumulated over several mini-batches before performing an optimization step.
- **Learning Rate Scheduler**: A learning rate scheduler could be added to decrease the learning rate as training progresses, often improving convergence.

This loop demonstrates an efficient and structured way of training a model with memory capabilities like KNN, using segmented sequences to ensure the model can handle large inputs. The validation step ensures that the model is evaluated periodically to prevent overfitting and monitor progress.

In [ ]:
model = MemorizingTransformer(embedding_dimension = 128,
                              vocab_size = 128,
                              max_knn_memories = MAX_KNN_MEMORIES)

model.to(device) ###########

optim = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
model.train()

for i in tqdm.tqdm(range(200), mininterval = 10., desc = 'training'):

    model.train()
    train_loss = 0.
    # Clear XL memories
    xl_memories = None
    # Clear KNN memory
    model.knn.clear()

    data = next(train_loader).to(device=device)
    seq, labels = data[:, :-1], data[:, 1:]

    t0 = time.time()
    print ("Begin document")

    # Each pass will be (BATCH_SIZE * SEGMENTS) iterations
    for seq_segment, labels_segment in zip(seq.chunk(SEGMENTS, dim = -1), labels.chunk(SEGMENTS, dim = -1)):

        loss, xl_memories = model(
            seq_segment,
            labels = labels_segment,
            xl_memories = xl_memories
        )

        train_loss += loss.item() / SEGMENTS
        (loss / SEGMENTS).backward()


    print(f'training loss: {train_loss}')
    t1 = time.time()
    print ("End document, total time:", t1 - t0)
    torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_CLIP_NORM)
    optim.step()
    optim.zero_grad()


    if not (i % VALIDATE_EVERY):
        model.eval()

        valid_data = next(val_loader)
        valid_loss = 0.

        with torch.no_grad():
            xl_memories = None
            model.knn.clear()
            seq, labels = data[:, :-1], data[:, 1:]

            for seq_segment, labels_segment in zip(seq.chunk(SEGMENTS, dim = -1), labels.chunk(SEGMENTS, dim = -1)):

                loss, xl_memories = model(
                    seq_segment,
                    labels = labels_segment,
                    xl_memories = xl_memories
                )

                valid_loss += loss.item() / SEGMENTS

        print(f'valid loss: {valid_loss}')


training:   0%|          | 0/200 [00:00<?, ?it/s]

Begin document
Begin KNN operations
End KNN operations, time taken: 0.4727809429168701
Begin KNN operations
End KNN operations, time taken: 0.7829794883728027
Begin KNN operations
End KNN operations, time taken: 0.9652132987976074
Begin KNN operations
End KNN operations, time taken: 1.1984717845916748
Begin KNN operations
End KNN operations, time taken: 1.5844461917877197
Begin KNN operations
End KNN operations, time taken: 1.8131227493286133
Begin KNN operations
End KNN operations, time taken: 1.985107183456421
Begin KNN operations
End KNN operations, time taken: 2.0774049758911133
Begin KNN operations
End KNN operations, time taken: 2.413853883743286
training loss: 5.044507646560668
End document, total time: 16.55513596534729
Begin KNN operations
End KNN operations, time taken: 0.4018399715423584
Begin KNN operations
End KNN operations, time taken: 0.6185629367828369
Begin KNN operations
End KNN operations, time taken: 0.8297295570373535
Begin KNN operations
End KNN operations, time 

training:   0%|          | 1/200 [00:30<1:40:42, 30.36s/it]

End KNN operations, time taken: 2.240504741668701
valid loss: 4.508983087539673
Begin document
Begin KNN operations
End KNN operations, time taken: 0.5132331848144531
Begin KNN operations
End KNN operations, time taken: 0.7702808380126953
Begin KNN operations
End KNN operations, time taken: 0.9941208362579346
Begin KNN operations
End KNN operations, time taken: 1.1803114414215088
Begin KNN operations
End KNN operations, time taken: 1.409938097000122
Begin KNN operations
End KNN operations, time taken: 1.6306748390197754
Begin KNN operations
End KNN operations, time taken: 1.9200448989868164
Begin KNN operations
End KNN operations, time taken: 2.0634706020355225
Begin KNN operations


training:   1%|          | 2/200 [00:45<1:09:48, 21.16s/it]

...

End KNN operations, time taken: 2.406026840209961
training loss: 2.4486013174057
End document, total time: 14.846877336502075
Begin document
Begin KNN operations
End KNN operations, time taken: 0.49159955978393555
Begin KNN operations
End KNN operations, time taken: 0.7573680877685547
Begin KNN operations
End KNN operations, time taken: 0.9809677600860596
Begin KNN operations
End KNN operations, time taken: 1.2288784980773926
Begin KNN operations
End KNN operations, time taken: 1.5425071716308594
Begin KNN operations
End KNN operations, time taken: 1.705092430114746
Begin KNN operations
End KNN operations, time taken: 1.929915428161621
Begin KNN operations
End KNN operations, time taken: 2.1608359813690186
Begin KNN operations


training: 100%|█████████▉| 199/200 [49:59<00:15, 15.05s/it]

End KNN operations, time taken: 2.4031240940093994
training loss: 2.50151789188385
End document, total time: 14.913466453552246
Begin document
Begin KNN operations
End KNN operations, time taken: 0.49132657051086426
Begin KNN operations
End KNN operations, time taken: 0.7613787651062012
Begin KNN operations
End KNN operations, time taken: 0.9866693019866943
Begin KNN operations
End KNN operations, time taken: 1.256568431854248
Begin KNN operations
End KNN operations, time taken: 1.4531452655792236
Begin KNN operations
End KNN operations, time taken: 1.6854360103607178
Begin KNN operations
End KNN operations, time taken: 1.9294054508209229
Begin KNN operations
End KNN operations, time taken: 2.147642135620117
Begin KNN operations


training: 100%|██████████| 200/200 [50:14<00:00, 15.07s/it]

End KNN operations, time taken: 2.4479188919067383
training loss: 2.3589978218078613
End document, total time: 14.891256093978882
